In [ ]:
# Parte 1: Análisis de Ventas de una Tienda
# Carga y Limpieza de Datos
# Comenzarás con tres conjuntos de datos con algunos problemas. Tu primera tarea es cargarlos, 
# unirlos y realizar una limpieza exhaustiva para dejarlos listos para el análisis.

import pandas as pd
import numpy as np

datos_ventas = {
    'Venta_ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10], # Hay una fila duplicada
    'Fecha': ['2023-01-01', '2023-01-01', '2023-01-02', '2023-01-02', '2023-01-03', '2023-01-08', '2023-01-08', '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-11'],
    'Producto_ID': [101, 102, 101, 103, 104, 101, 102, 103, 102, 101, 101],
    'Vendedor_ID': [1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1],
    'Cantidad': [20, 5, 25, 7, 10, 18, 6, 8, np.nan, 15, 15], # Hay un valor nulo
    'Precio_str': ['$ 5.00', '€ 20.50', '$ 5.50', '$ 8.00', '€ 3.00', '$ 5.25', '$ 21.00', '$ 8.50', '€ 22.00', '$ 5.00', '$ 5.00'],
    'Sucursal': ['Norte', '  Sur', 'norte', 'Norte', 'Sur', 'Norte', 'Sur', 'Norte', 'Sur', '  norte  ', '  norte  '] # Nombres inconsistentes
}

datos_productos = {
    'Producto_ID': [101, 102, 103, 104],
    'Nombre_Producto': ['Café', 'Tarta', 'Pastel', 'Té'],
    'Categoría': ['Bebida', 'Postre', 'Postre', 'Bebida']
}

datos_vendedores = {
    'Vendedor_ID': [1, 2],
    'Nombre_Vendedor': ['Ana', 'Luis']
}

# Crea un DataFrame para cada diccionario.

datos_ventas_df = pd.DataFrame(datos_ventas)
datos_productos_df = pd.DataFrame(datos_productos)
datos_vendedores_df = pd.DataFrame(datos_vendedores)

# Limpia `df_ventas`:

# Elimina las filas completamente duplicadas. (Pista: investiga el método .drop_duplicates()).

datos_ventas_df = datos_ventas_df.drop_duplicates()
datos_productos_df = datos_productos_df.drop_duplicates()
datos_vendedores_df = datos_vendedores_df.drop_duplicates()

# Maneja el valor nulo en `Cantidad`: elimina cualquier fila que contenga valores nulos. (Pista: investiga el método .dropna()).

datos_ventas_df = datos_ventas_df.dropna(subset=['Cantidad'])

# Normaliza la columna `Sucursal`: conviértela a minúsculas y elimina los espacios en blanco al inicio y al final.

datos_ventas_df['Sucursal'] = datos_ventas_df['Sucursal'].str.lower().str.strip()
# print(datos_ventas_df)

# Limpia la columna `Precio_str`: esta columna contiene múltiples divisas ('$' y '€') y espacios. Debes eliminar todos los 
# caracteres no numéricos (excepto el punto decimal) y convertir la columna a un tipo numérico `float`.

#datos_ventas_df['Precio_str'] = datos_ventas_df['Precio_str'].str.replace(' ', '').str.replace('$', '').str.replace('€', '').astype(float)
# La linea comentada no tenía en cuenta todos los carácteres no numéricos, la nueva si

# Elimina todos los caracteres que no sean dígitos o punto decimal en 'Precio_str', y convierte el resultado a número float.
datos_ventas_df['Precio_str'] = datos_ventas_df['Precio_str'].str.replace(r'[^\d.]', '', regex=True).astype(float)
# print(datos_ventas_df)

# Une los tres DataFrames en uno solo que contenga toda la información.

union_ventas_productos = pd.merge(datos_ventas_df, datos_productos_df, on='Producto_ID', how='outer')
union_ventas_productos_vendedores = pd.merge(union_ventas_productos, datos_vendedores_df, on='Vendedor_ID', how='outer')

# Crea una nueva columna `Ingresos_Totales` multiplicando el precio por la cantidad.

union_ventas_productos_vendedores['Ingresos_Totales'] = union_ventas_productos_vendedores.apply(lambda fila: (fila['Precio_str'] * fila['Cantidad']), axis=1)
# print(union_ventas_productos_vendedores)

# Convierte la columna `Fecha` a formato `datetime` y establécela como el índice del DataFrame final. 

union_ventas_productos_vendedores['Fecha'] = pd.to_datetime(union_ventas_productos_vendedores['Fecha'])
union_ventas_productos_vendedores = union_ventas_productos_vendedores.set_index('Fecha')

# Puedes usar .head() para inspeccionar el resultado.

union_ventas_productos_vendedores.head()
#print(union_ventas_productos_vendedores)

# Análisis Exploratorio y Creación de Variables
# Con los datos limpios, realizarás un análisis para responder a preguntas clave de negocio y crearás nuevas variables.

# Muestra por pantalla los ingresos totales por `Categoría` y por `Nombre_Vendedor`.

ingresos_totales_categoria_nombre_vendedor = union_ventas_productos_vendedores.groupby(['Categoría', 'Nombre_Vendedor'])['Ingresos_Totales'].sum()
# print(ingresos_totales_categoria_nombre_vendedor)

# Muestra ahora la cantidad total de productos vendidos por `Categoría` (índice) en cada `Sucursal` (columnas).

cantidad_productos_vendidos_categoria_sucursal = union_ventas_productos_vendedores.pivot_table(values = 'Cantidad', index = 'Categoría', columns = 'Sucursal', aggfunc='sum', margins=True, margins_name='Total Cantidad', fill_value = 0)
# print(cantidad_productos_vendidos_categoria_sucursal)

# Crea una nueva columna categórica llamada `Tipo_Venta`. Si `Ingresos_Totales` es mayor que $50, el valor será 'Grande'; 
# de lo contrario, será 'Normal'.

def tipo_de_venta(precio):
    if(precio>50):
        return 'Grande'
    else:
        return 'Normal'
    
union_ventas_productos_vendedores['Tipo_Venta'] = union_ventas_productos_vendedores['Ingresos_Totales'].apply(tipo_de_venta)
#print(union_ventas_productos_vendedores)

# Desafío - Producto Estrella: Para cada `Categoría`, encuentra cuál fue el `Nombre_Producto` que generó más `Ingresos_Totales`. 
# (Pista: investiga cómo una vez agrupados ciertos datos, se puede el valor máximo en cada grupo).

nombre_producto_con_mas_ingresos = union_ventas_productos_vendedores.groupby('Categoría')['Ingresos_Totales'].idxmax()
filas_max_ingresos = union_ventas_productos_vendedores.loc[nombre_producto_con_mas_ingresos, ['Categoría', 'Nombre_Producto', 'Ingresos_Totales']]

# print(filas_max_ingresos)

# Desafío - Contribución al Ingreso: Calcula una nueva columna llamada `Pct_Ingreso_Categoria` que muestre qué porcentaje 
# representan los `Ingresos_Totales` de cada venta con respecto al total de ingresos de su `Categoría`. 
# (Pista: para este cálculo, donde necesitas un valor agregado (la suma por categoría) pero 
# quieres mantener la forma original del DataFrame, investiga el método .transform()).

union_ventas_productos_vendedores['Pct_Ingreso_Categoria'] = (union_ventas_productos_vendedores['Ingresos_Totales'] / union_ventas_productos_vendedores.groupby('Categoría')['Ingresos_Totales'].transform('sum') * 100).round(2)
# print(union_ventas_productos_vendedores)

# Análisis Temporal
# Investigarás cómo varían las ventas a lo largo del tiempo para identificar patrones.
# Utiliza `.resample()` para calcular los ingresos totales diarios ('D') y semanales ('W').

ingresos_diarios = union_ventas_productos_vendedores['Ingresos_Totales'].resample('D').sum()
ingresos_semanales = union_ventas_productos_vendedores['Ingresos_Totales'].resample('W').sum()

print(ingresos_diarios)
print(ingresos_semanales)

# Encuentra el día de la semana (Lunes, Martes, etc.) con el promedio de ingresos más alto. 
# (Pista: después de agrupar, puedes usar .sort_values() para ordenar los resultados).

dias_ingles_espanol = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

# print(union_ventas_productos_vendedores)
union_ventas_productos_vendedores['Dia_Semana'] = union_ventas_productos_vendedores.index.day_name().map(dias_ingles_espanol)
# El índice es la fecha, por eso no hace falta concretarla, si lo hiciera, no funcionaría, ya que no deja
# referenciar al índice

dia_mejor_promedio = union_ventas_productos_vendedores.groupby('Dia_Semana')['Ingresos_Totales'].mean()
dia_mejor_promedio_sorted = dia_mejor_promedio.sort_values()

dia_max = dia_mejor_promedio_sorted.idxmax()
promedio_max = dia_mejor_promedio_sorted.max()

print(f"El día con el mayor promedio de ingresos es {dia_max} con un promedio de {promedio_max:.2f}")



Fecha
2023-01-01    202.5
2023-01-02    193.5
2023-01-03     30.0
2023-01-04      0.0
2023-01-05      0.0
2023-01-06      0.0
2023-01-07      0.0
2023-01-08    220.5
2023-01-09     68.0
2023-01-10      0.0
2023-01-11     75.0
Freq: D, Name: Ingresos_Totales, dtype: float64
Fecha
2023-01-01    202.5
2023-01-08    444.0
2023-01-15    143.0
Freq: W-SUN, Name: Ingresos_Totales, dtype: float64
Domingo


In [80]:
# Parte 2: Análisis Exploratorio de Datos de Inmuebles
# Contexto y Datos Iniciales
# En esta segunda parte, actuarás como un analista de datos para una agencia inmobiliaria. 
# Te han proporcionado un conjunto de datos con propiedades en venta en diferentes ciudades y tu tarea 
# es realizar un análisis exploratorio para extraer información valiosa para la empresa y sus clientes.

datos_inmuebles = {
    'Referencia': ['VLC-01', 'MAD-01', 'BCN-01', 'VLC-02', 'MAD-02', 'BCN-02', 'VLC-03', 'MAD-03', 'BCN-03', 'VLC-04'],
    'Ciudad': ['Valencia', 'Madrid', 'Barcelona', 'Valencia', 'Madrid', 'Barcelona', 'Valencia', 'Madrid', 'Barcelona', 'Valencia'],
    'Habitaciones': [3, 4, 2, 4, 5, 3, 2, 3, 4, 3],
    'Baños': [2, 2, 1, 3, 3, 2, 1, 2, 2, 2],
    'Superficie': [90, 120, 65, 110, 150, 85, 55, 95, 115, 100],
    'Precio': [210000, 450000, 320000, 280000, 600000, 410000, 180000, 390000, 550000, 250000],
    'Tipo_Propiedad': ['Piso', 'Piso', 'Apartamento', 'Piso', 'Chalet', 'Piso', 'Apartamento', 'Piso', 'Piso', 'Piso'],
    'Extras': ['Garaje', 'Piscina', np.nan, 'Garaje;Terraza', 'Piscina;Jardín', 'Terraza', np.nan, 'Garaje', 'Garaje;Piscina', 'Terraza']
}

datos_inmuebles_df = pd.DataFrame(datos_inmuebles)

# Preguntas de Negocio a Resolver
# A partir del DataFrame, deberás aplicar las técnicas de Pandas que consideres necesarias para preparar 
# los datos y responder a las siguientes preguntas. 
# El enunciado es abierto y te dará pocas pistas sobre cómo avanzar. Deberás justificar cada paso que realices.

# Preparación de Datos:
# ¿Cómo podrían gestionarse los valores nulos en la columna `Extras`?. Propón y aplica una solución.

# datos_inmuebles_df = datos_inmuebles_df.dropna(subset=['Extras'])
# El dropna podría ser una opción, pero así se eliminaría la fila que contenga el apartado de Extras vacío,
# hay opciones mejores

datos_inmuebles_df['Extras'] = datos_inmuebles_df['Extras'].fillna('Sin Extras')
# Esta sería la mejor opción, ya que con fillna, en caso de estar vacío ese campo, solamente se pone "Sin Extras"

# La columna `Extras` contiene varios valores en una sola cadena de texto. Adopta una solución para poder analizar los extras de forma individual.

datos_inmuebles_df['Extras'] = datos_inmuebles_df['Extras'].str.split(';')
# La mejor forma para trabajar con varios valores en una cadena sería usar el split, buscar el valor que realiza la separación
# y guardarlos en un array, así se podría revisar sin problemas cada cosa

# print(datos_inmuebles_df)

# Crea una nueva variable que consideres relevante para el análisis (por ejemplo, el precio por metro cuadrado).

datos_inmuebles_df['Precio_m2'] = (datos_inmuebles_df['Precio'] / datos_inmuebles_df['Superficie']).round(2)
# print(datos_inmuebles_df)

# Análisis del Mercado:
# ¿Cuál es la distribución de precios y superficies en las diferentes ciudades? ¿Qué ciudad es, en promedio, más cara?

distribucion_ciudades_precios_superficies = datos_inmuebles_df.groupby('Ciudad')[['Precio', 'Superficie']].agg(['mean', 'max']).round(2)
# print(distribucion_ciudades_precios_superficies)

# ¿Cómo dirías que influye el número de habitaciones y baños en el precio final de las viviendas en cada ciudad?

numero_habitaciones_baños_ciudad = datos_inmuebles_df.groupby(['Habitaciones', 'Baños', 'Ciudad'])['Precio'].mean()
# print(numero_habitaciones_baños_ciudad)

# Según la media que se puede ver que da el resultado, podemos observar que 
# Barcelona y Madrid de por si son más caras, y que si tiene más habitaciones y cuartos de baño, sube el precio

# ¿Qué `Tipo_Propiedad` (Piso, Chalet, etc.) tiene un mayor precio medio? ¿Y una mayor superficie media?

# Precio medio por tipo de propiedad
precio_medio_por_tipo = datos_inmuebles_df.groupby('Tipo_Propiedad')['Precio'].mean()
tipo_mayor_precio = precio_medio_por_tipo.idxmax()
mayor_precio = precio_medio_por_tipo.max()

# Superficie media por tipo de propiedad
superficie_media_por_tipo = datos_inmuebles_df.groupby('Tipo_Propiedad')['Superficie'].mean()
tipo_mayor_superficie = superficie_media_por_tipo.idxmax()
mayor_superficie = superficie_media_por_tipo.max()

#print(f"El tipo de propiedad con mayor precio medio es {tipo_mayor_precio} con un precio medio de {mayor_precio:.2f}")
#print(f"El tipo de propiedad con mayor superficie media es {tipo_mayor_superficie} con una superficie media de {mayor_superficie:.2f} m²")

# ¿Qué porcentaje de las propiedades en cada ciudad ofrece 'Piscina'? ¿Y 'Garaje'?

datos_inmuebles_df['Tiene_Piscina'] = datos_inmuebles_df['Extras'].apply(lambda extra: 'Piscina' in extra)
datos_inmuebles_df['Tiene_Garaje'] = datos_inmuebles_df['Extras'].apply(lambda extra: 'Garaje' in extra)

porcentaje_por_ciudad = datos_inmuebles_df.groupby('Ciudad')[['Tiene_Piscina', 'Tiene_Garaje']].mean() * 100
print(porcentaje_por_ciudad)

# Segmentación y Recomendación:
# Identifica las 3 propiedades con la mejor relación superficie-precio. (Pista: utiliza .sort_values() para ordenar y .head(3) para seleccionar las primeras).

mejor_relacion_precio = datos_inmuebles_df.sort_values('Precio_m2').head(3)
print(mejor_relacion_precio)

# Filtra y muestra un listado de propiedades que podrían ser de interés para una familia (ej: más de 3 habitaciones y con garaje), 
# ordenadas de menor a mayor precio. (Pista: no olvides usar .sort_values() al final).

propiedades_interesantes = datos_inmuebles_df[datos_inmuebles_df.apply(lambda fila: (fila['Habitaciones'] >=3) and (fila['Tiene_Garaje'] == True) ,axis=1)].sort_values('Precio')
print(propiedades_interesantes)


           Tiene_Piscina  Tiene_Garaje
Ciudad                                
Barcelona      33.333333     33.333333
Madrid         66.666667     33.333333
Valencia        0.000000     50.000000
  Referencia    Ciudad  Habitaciones  Baños  Superficie  Precio  \
0     VLC-01  Valencia             3      2          90  210000   
9     VLC-04  Valencia             3      2         100  250000   
3     VLC-02  Valencia             4      3         110  280000   

  Tipo_Propiedad             Extras  Precio_m2  Tiene_Piscina  Tiene_Garaje  
0           Piso           [Garaje]    2333.33          False          True  
9           Piso          [Terraza]    2500.00          False         False  
3           Piso  [Garaje, Terraza]    2545.45          False          True  
  Referencia     Ciudad  Habitaciones  Baños  Superficie  Precio  \
0     VLC-01   Valencia             3      2          90  210000   
3     VLC-02   Valencia             4      3         110  280000   
7     MAD-03     Mad